<a href="https://colab.research.google.com/github/urstrulydeva/PythonProjectDev/blob/main/PythonMiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torchvision
import numpy as np

# header files needed
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [ ]:
class InceptionBlock(nn.Module):
    def _init_(self, in_channels, out1x1, red3x3, out3x3, red5x5, out5x5, maxpool3x3):
        super(InceptionBlock, self)._init_()

        self.branch1 = ConvBlock(in_channels, out1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, red3x3, kernel_size=1),
            ConvBlock(red3x3, out3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, red5x5, kernel_size=1),
            ConvBlock(red5x5, out5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels, maxpool3x3, kernel_size=1)
        )

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)

In [ ]:
class ConvBlock(nn.Module):
    def _init_(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self)._init_()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.relu = nn.ReLU()
        self.batchNorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        return self.relu(self.batchNorm(self.conv(x)))

In [ ]:
class ResNet50Net(nn.Module):
    def _init_(self, num_classes=26):
        super(ResNet50Net, self)._init_()
        resnet = models.resnet50(pretrained=True)  # Load the pre-trained ResNet-50 model
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # Remove the last fully connected layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Adapt the spatial dimensions before the fully connected layer
        self.fc = nn.Linear(2048, num_classes)  # Update the input size to match the flattened feature map

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        print("After flattening:", x.shape)  # Add this print statement

        x = self.fc(x)
        return x

In [ ]:
import torchvision.models as models

In [50]:
if __name__ == '__main__':
    # Initialize InceptionNet model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_classes = 6
    model = models.resnet50(pretrained=True)  # You can use a pretrained ResNet model
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_classes)  # Replace the final fully connected layer
    model = model.to(device)

In [52]:
transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [53]:
train_data = datasets.ImageFolder("/content/drive/My Drive/Trainingdata_final/", transform=transform)
val_data = datasets.ImageFolder("/content/drive/My Drive/Validationdata/", transform=transform)

In [54]:
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
train_loader=DataLoader(train_data, batch_size=64,shuffle=False)

In [55]:
for inputs, labels in train_loader:
    print("Training data shape:", inputs.shape)
    print("Training label shape:", labels.shape)
    break

for inputs, labels in val_loader:
    print("Validation data shape:", inputs.shape)
    print("Validation label shape:", labels.shape)
    break

Training data shape: torch.Size([64, 3, 299, 299])
Training label shape: torch.Size([64])
Validation data shape: torch.Size([64, 3, 299, 299])
Validation label shape: torch.Size([64])


In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(num_classes), lr=0.001)

In [57]:
for inputs, labels in train_loader:
    print(inputs.shape)
    break  # Print the shape of the first batch only

torch.Size([64, 3, 299, 299])


In [1]:
# Training loop
num_epochs = 10 # Modify this to the desired number of epochs
for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        model.eval()
        val_loss = 0.0
        corrects = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                corrects += (predicted == labels).sum().item()

        epoch_loss = train_loss / len(train_data)
        epoch_val_loss = val_loss / len(val_data)
        accuracy = 100.0 * corrects / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Accuracy: {accuracy:.2f}%")

    # Save the trained model
torch.save(model.state_dict(), "/content/drive/My Drive/inceptionmodel/Resnet_save_model.pt")
print("The model is saved at /content/drive/My Drive/inceptionmodel/save_model.pt ")

NameError: ignored

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Step 1: Load the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ResNet50Net(num_classes=6).to(device)
model.load_state_dict(torch.load("/content/drive/My Drive/inceptionmodel/Resnet_save_model.pt"))
model.eval()  # Set the model to evaluation mode

# Step 2: Prepare the test data
test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_data = datasets.ImageFolder("/content/drive/My Drive/Testingdata", transform=test_transform)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

# Step 3: Test the model
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # Visualize the images along with their predicted labels
        for i in range(images.size(0)):
            image = images[i].cpu().numpy().transpose(1, 2, 0)  # Convert to (H, W, C) format
            plt.imshow(image)
            plt.title(f"Predicted Label: {predicted[i].item()}, True Label: {labels[i].item()}")
            plt.show()

accuracy = 100.0 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have already loaded and defined your model and test data
# If not, you can load the model and test data similar to the training data

# Define the test data loader (assuming you have a variable named test_data containing the test dataset)
batch_size = 1  # Set the batch size to 1 for getting predictions for each individual sample
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Set the model in evaluation mode
model.eval()

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test data and make predictions
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)  # Move data to GPU if available
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calculate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix as a colorful image
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()